# Necessary Common Functions

Those functions should be ran before each part.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import numba 
from numba import jit
import datasets
from datasets import Dataset as TFDataset
import multiprocessing

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_sessions(df: pd.DataFrame, test=False, list_item=False) -> list:
    
    all_item = []
    if 'next_item' in df and not test:
        if list_item:
            for i in trange(len(df)):
                all_item.append(np.concatenate([np.array(df.loc[i, 'prev_items']), np.array(df.loc[i, 'next_item'])], axis=0))
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items'][:-1]+f" '{df.loc[i, 'next_item']}']").replace(" ", ",")))
    else:
        if list_item:
            all_item = df['prev_items']
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items']).replace(" ", ",")))
    return all_item

In [ ]:
def get_co_occurence_dict(sessions: list, bidirection: bool=True, weighted: bool=False, max_dis=None) -> dict:
    res = {}
    for sess in tqdm(sessions):
        for i, id in enumerate(sess):
            if id not in res:
                res[id] = Counter()
            
            if max_dis == None:
                e = len(sess)
            else:
                e = min(i + max_dis + 1, len(sess))

            for j in range(i+1, e):
                if not weighted:
                    res[id][sess[j]] += 1
                else:
                    res[id][sess[j]] += 1 / (j-i)
                if bidirection:
                    if sess[j] not in res:
                        res[sess[j]] = Counter()
                    if not weighted:
                        res[sess[j]][id] += 1
                    else:
                        res[sess[j]][id] += 1 / (j-i)
    return res

In [ ]:
def sort_co_occurence_dict(co_occurence_dict: dict) -> dict:
    res = {}
    for k,v in co_occurence_dict.items():
        res[k] = dict(sorted(v.items(), key=lambda item: -item[1]))
    return res

In [ ]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [ ]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

In [ ]:
def get_co_graph_counts(session_last_items, merged_candidates_df, co_graph_dict):
    co_graph_count_list = []
    for idx, row in tqdm(merged_candidates_df.iterrows(), total=merged_candidates_df.shape[0]):
        sess_id = row['sess_id']
        product = row['product']
        last_item = session_last_items[sess_id]
        co_graph_count = co_graph_dict[last_item][product]
        co_graph_count_list.append(co_graph_count)
    return co_graph_count_list

In [ ]:
def flatten_co_graph_dict(co_graph_dict):
    product_list = []
    neighbor_list = []
    counts_list = []
    for product in tqdm(co_graph_dict.keys(), total=len(co_graph_dict)):
        for neigh in co_graph_dict[product].keys():
            product_list.append(product)
            neighbor_list.append(neigh)
            counts_list.append(co_graph_dict[product][neigh])
    return pd.DataFrame({'product_' : product_list, 'neighbor' : neighbor_list, 'counts' : counts_list})

In [ ]:
def normalize_co_graph_counts(merged_candidates_counts):
    # normalize co graph counts 
    # merged_candidates_counts_g = cudf.from_pandas(merged_candidates_counts)
    sessions_count_sum = merged_candidates_counts[['sess_id', 'counts']].groupby('sess_id').sum()
    sessions_count_sum.sort_index(inplace=True)

    sessions_count_sum = sessions_count_sum.to_pandas()

    candidates_count_sum = sessions_count_sum.loc[merged_candidates_counts['sess_id']].reset_index(drop=True)
    merged_candidates_counts['counts_cum'] = candidates_count_sum['counts']
    merged_candidates_counts['normalized_counts'] = merged_candidates_counts['counts'] / merged_candidates_counts['counts_cum']

    # del merged_candidates_counts_g
    # del sessions_count_sum_g
    

# Merge valid co-graph counts 

In [ ]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_no_hist_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [ ]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(test_sessions_path)

In [ ]:
merged_candidates_feature = read_merged_candidates_feature()

In [ ]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [ ]:
train_sess_data = read_train_data()
valid_sess_data = read_valid_data()
test_sess_data = read_test_data()
product = read_product_data()

In [ ]:
train_sess_item = get_sessions(train_sess_data, list_item=False)
valid_sess_item = get_sessions(valid_sess_data, test=True, list_item=False)

100%|██████████| 361581/361581 [00:12<00:00, 29670.48it/s]


## bidirection

In [ ]:
# bidirection
co_occurence_dict_bi = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False)

In [ ]:
# only one arg, can't use another arg
def get_bi_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_bi and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_bi[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 

    return cand_counter

In [ ]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [ ]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
bi_valid_sessions_counter = pool.map(get_bi_valid_session_co_graph_candidates, sess_ids)

In [ ]:
valid_co_graph_candidates, len(bi_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [ ]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(bi_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list

100%|██████████| 84407339/84407339 [03:18<00:00, 424750.81it/s]
/tmp/ipykernel_92972/4091948514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list


In [ ]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_0'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']

/tmp/ipykernel_92972/860000090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_92972/860000090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']


In [ ]:
merged_candidates_feature['normalized_all_items_co_graph_count_0'] = merged_candidates['normalized_all_items_co_graph_count_0']
merged_candidates_feature['all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0']

In [ ]:
merged_candidates_feature.query('sess_id==20000').sort_values(by=['sasrec_normalized_scores_3'], ascending=False)[['product', 'sasrec_normalized_scores_3', 'all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0']][:25]

,product,sasrec_normalized_scores_3,all_items_co_graph_count_0,normalized_all_items_co_graph_count_0
4672245,B07D98ZNJX,8.362947e-01,11,0.250000
4672246,B07D997JWG,1.596784e-01,10,0.227273
4672267,B07QJB4MFD,2.989717e-03,4,0.090909
4672174,B007SQ3HGS,3.628744e-04,2,0.045455
4672394,B09Y5GC5R7,3.485525e-04,2,0.045455
4672283,B07V3P7Z9R,1.137960e-04,2,0.045455
4672297,B081N7R51C,1.123027e-04,1,0.022727
4672300,B081N8XPQH,6.985810e-05,0,0.000000
4672284,B07V4PBMWT,1.069833e-05,2,0.045455
4672340,B08SVPH33Z,9.188924e-06,3,0.068182


In [ ]:
cast_dtype(merged_candidates_feature, 
           ['all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0'])
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')